# **Security Course**
### <span style="font-weight:bold;">Instructor:</span> <span style="font-size:19px;"><a href="https://ir.linkedin.com/in/hrshahriari">Dr. Hamid Reza Shahriari</a></span>
### <span style="font-weight:bold;">Organization</span>
### <span style="font-size:19px;"><a href="https://aut.ac.ir/en">Amirkabir University of Technology (Tehran Polytechnic)</a></span>
<img src="../assets/AKUT-white-logo.png" alt="Amirkabir University of Technology logo" style="width:300px;">

# **Diffi-Hellman Algorithm**
In this notebook, we will walk through the steps of implementing the Diffi-Hellman algorithm.

### Import

In [74]:
import socket
import random
import threading
from time import sleep
from utils import mod_exp

%run ./utils.ipynb

### Create Algorithm Paramters

In [75]:
q=int(input("Enter a prime number: "))
while(True):
    if is_prime(q):
        break
    print("Input number is not prime, try again!")

alpha = find_primitive_root(q)
print(f"selected primitve root or \u03B1 for your prime number is {alpha}.")

selected primitve root or α for your prime number is 134.


### Alice Side

In [76]:
def Alice_side():
    q = 137
    alpha = 134

    print(f"--Alice Side-- Hi Alice! \u03B1={alpha} and q={q} :).")
    print("--Alice Side-- I'm trying to create a random number for you (as your private key)!")
    sleep(2)
    X_Alice= random.randint(1, q-1)
    print(f"--Alice Side-- Your private key is {X_Alice}.")

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('localhost', 12345))
    server.listen(1)

    conn, _ = server.accept()

    Y_Alice = mod_exp(alpha, X_Alice, q)
    print("--Alice Side-- I will send your public server value Y to Bob.")

    conn.send(str(Y_Alice).encode())

    Y_Bob = int(conn.recv(1024).decode())
    sleep(1)
    print(f"--Alice Side-- I send your Y to Bob and he sends this for you: Y_Bob= {Y_Bob}")

    shared_secret = mod_exp(Y_Bob, X_Alice, q)
    print(f"--Alice Side-- Your shared secret: {shared_secret}")

    server.close()

### Bob Side

In [77]:
def Bob_side():
    q = 137
    alpha = 134

    print(f"--Bob Side-- Hi Bob! \u03B1={alpha} and q={q} :).")
    print("--Bob Side-- I'm trying to create a random number for you (as your private key)!")
    sleep(2)
    X_Bob= random.randint(1, q-1)
    print(f"--Bob Side-- Your private key is {X_Bob}.")

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client.connect(('localhost', 12345))

    Y_Bob = mod_exp(alpha, X_Bob, q)
    print("--Bob Side-- I will send your public server value Y to Alice.")

    Y_Alice = int(client.recv(1024).decode())
    sleep(1)
    print(f"--Bob Side-- I send your Y to Alice and she sends this for you: Y_Alice= {Y_Alice}")

    client.send(str(Y_Bob).encode())

    shared_secret = mod_exp(Y_Alice, X_Bob, q)
    print(f"--Bob Side-- Your shared secret: {shared_secret}")

    client.close()


### Run Alice and Bob codes

In [78]:
server_thread = threading.Thread(target=Alice_side)
client_thread = threading.Thread(target=Bob_side)
server_thread.start()
client_thread.start()
try:
    server_thread.join()
    sleep(1)
    client_thread.join()
except:
    pass

--Alice Side-- Hi Alice! α=134 and q=137 :).
--Alice Side-- I'm trying to create a random number for you (as your private key)!
--Bob Side-- Hi Bob! α=134 and q=137 :).
--Bob Side-- I'm trying to create a random number for you (as your private key)!
--Bob Side-- Your private key is 65.
--Alice Side-- Your private key is 69.
--Bob Side-- I will send your public server value Y to Alice.
--Alice Side-- I will send your public server value Y to Bob.
--Bob Side-- I send your Y to Alice and she sends this for you: Y_Alice= 3
--Bob Side-- Your shared secret: 71
--Alice Side-- I send your Y to Bob and he sends this for you: Y_Bob= 66
--Alice Side-- Your shared secret: 71


# **Attachment: Man In The Middle Attack to Diffi-Hellman Algorithm**

In [79]:
# TODO

# **Attachment: Authenticated Diffi-Hellman Algorithm**

### Imports

In [80]:
import socket
import random
import threading
from time import sleep
from utils import mod_exp

%run ./utils.ipynb

### Create Algorithm Parameters

In [81]:
g=2

n_Bob=15
e_Bob=7
d_Bob=15

n_Alice=33
e_Alice=3
d_Alice=7

MAX_RAND_NUMBER= 3

### Alice Side

In [82]:
def Alice_side_authenticated():
    Alice_private_key= (d_Alice, n_Alice)
    Bob_public_key= (e_Bob, n_Bob)

    print(f"--Alice Side-- Hi Alice! your private key is {Alice_private_key} and Bob public key is {Bob_public_key}:).")
    print("--Alice Side-- I'm trying to create a random number for you (as your private key)!")
    sleep(2)
    Alice_rand_numb= random.randint(1, MAX_RAND_NUMBER)
    X_Alice= mod_exp(g, Alice_rand_numb, None)
    print(f"--Alice Side-- Your private key is {Alice_rand_numb}, and g^(your private key) is {X_Alice}.")

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('localhost', 12345))
    server.listen(1)

    conn, _ = server.accept()
    sign_X_Alice = mod_exp(X_Alice, Alice_private_key[0], Alice_private_key[1])

    print(f"--Alice Side-- Your sign is equal to: {sign_X_Alice}.I will send your sing and g^(your private key) to Bob.")

    message = f"{sign_X_Alice},{X_Alice}"
    conn.send(str(message).encode())

    data = conn.recv(1024).decode()
    sing_X_Bob, X_Bob= map(int, data.split(','))
    sleep(1)

    print(f"--Alice Side-- Bob sends these for you: {sing_X_Bob, X_Bob}.")
    verify_result= verify(sing_X_Bob, X_Bob, Bob_public_key)

    if verify_result:
        print(f'--Alice Side-- I verify the sign and it was Bob :). Now continue the algorithm.')
    else:
        raise Exception(f'--Alice Side-- Verify faield so message does not sent by Bob...! I Think Eve is trying to hack you!')

    shared_secret= mod_exp(X_Bob, Alice_rand_numb, None)
    print(f"--Alice Side-- Your shared secret: {shared_secret}")

    conn.close()

### Bob Side

In [83]:
def Bob_side_authenticated():
    Bob_private_key= (d_Bob, n_Bob)
    Alice_public_key= (e_Alice, n_Alice)
    
    print(f"--Bob Side-- Hi Bob! your private key is {Bob_private_key} and Alice public key is {Alice_public_key}:).")
    print("--Bob Side-- I'm trying to create a random number for you (as your private key)!")
    sleep(2)
    Bob_rand_numb= random.randint(1, MAX_RAND_NUMBER)
    X_Bob= mod_exp(g, Bob_rand_numb, None)
    print(f"--Bob Side-- Your private key is {Bob_rand_numb}, and g^(your private key) is {X_Bob}.")

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client.connect(('localhost', 12345))

    data = client.recv(1024).decode()
    sing_X_Alice, X_Alice= map(int, data.split(','))
    sleep(1)
    print(f"--Bob Side-- Alice sends these for you: {sing_X_Alice, X_Alice}.")
    verify_result= verify(sing_X_Alice, X_Alice, Alice_public_key)
    if verify_result:
        print(f'--Bob Side-- I verify the sign and it was Alice :). Now continue the algorithm.')
    else:
        raise Exception(f'--Bob Side-- Verify faield so message does not sent by Alice...! I Think Eve is trying to hack you!')

    sign_X_Bob = mod_exp(X_Bob, Bob_private_key[0], Bob_private_key[1])

    print(f"--Bob Side-- Your sign is equal to: {sign_X_Bob}.I will send your sing and g^(your private key) to Alice.")

    message = f"{sign_X_Bob},{X_Bob}"
    client.send(str(message).encode())

    shared_secret= mod_exp(X_Alice, Bob_rand_numb, None)
    print(f"--Bob Side-- Your shared secret: {shared_secret}")

    client.close()

### Run Alice and Bob codes

In [84]:
server_thread = threading.Thread(target=Alice_side_authenticated)
client_thread = threading.Thread(target=Bob_side_authenticated)
server_thread.start()
client_thread.start()
try:
    server_thread.join()
    sleep(1)
    client_thread.join()
except:
    pass

--Alice Side-- Hi Alice! your private key is (7, 33) and Bob public key is (7, 15):).
--Alice Side-- I'm trying to create a random number for you (as your private key)!
--Bob Side-- Hi Bob! your private key is (15, 15) and Alice public key is (3, 33):).
--Bob Side-- I'm trying to create a random number for you (as your private key)!
--Alice Side-- Your private key is 2, and g^(your private key) is 4.
--Bob Side-- Your private key is 3, and g^(your private key) is 8.
--Alice Side-- Your sign is equal to: 16.I will send your sing and g^(your private key) to Bob.
--Bob Side-- Alice sends these for you: (16, 4).
--Bob Side-- I verify the sign and it was Alice :). Now continue the algorithm.
--Bob Side-- Your sign is equal to: 2.I will send your sing and g^(your private key) to Alice.
--Bob Side-- Your shared secret: 64
--Alice Side-- Bob sends these for you: (2, 8).
--Alice Side-- I verify the sign and it was Bob :). Now continue the algorithm.
--Alice Side-- Your shared secret: 64
